工作中，经常需要从一些Excel表中复制一些数据，填入统计表中。复制粘贴的体验确实太过机械，枯燥乏味。这种事交给Python，它会心甘情愿，毫无怨言。好了，上实例。领导要求每月将如下“原报表”中粉色部分的数据复制到“抽出数据报表”中。先观察一下“原报表”，这个报表的格式是固定的，不能更改。其中第8,9,11行是没有数据的，因此在提取数据的时候需要跳过。所以我们的步骤如下：
1. 提取“原报表”第8行往下不为空的数据
2. 写入到“抽出数据报表”
3. 设置写入内容的格式及边框、行高等，保存

![](images\problem.png)

因为原报表是".xls"后缀的，所以不能使用`openpyxl`库，这里使用`xlrd`。读取工作簿，存入`wb`，然后选取第一个工作表，存入`ws`。新建一个列表`data`用于储存获取的数据。然后从第8行（对应索引为0）遍历工作表。其中第2列（索引为1）对应为“科室名称”，第3列是“总计”，第4列是“计”，第21列是“平均每日人次”。将每行的数据放入列表`info_list`。由于有些行的数据是空的，所以加入一个`if`语句进行判断，只有不是空数据，才放入总列表`data`。最后查看一下`data`列表的数据，正是我们需要的。

In [11]:
#读取xls文件中的数据
import xlrd

file = "原表.xls"
wb = xlrd.open_workbook(file) #读取工作簿
ws = wb.sheets()[0] #选第一个工作表
data = []

for row in range(7, ws.nrows):
    name = ws.cell(row, 1).value.strip() #科室名称
    total1 = ws.cell(row, 2).value #总计
    total2 = ws.cell(row, 3).value #计
    avg = ws.cell(row, 20).value #平均每日人次
    
    info_list=[name,total1,total2,avg]

    if info_list[0] != "": #去除空数据
        data.append(info_list)
data 

[['A科', '1585', 1615.0, '53'],
 ['B科', 749.0, 742.0, '23'],
 ['C科', 549.0, 543.0, '17'],
 ['D科', 1608.0, 1604.0, '52'],
 ['E科', 471.0, 470.0, '14'],
 ['F科', 651.0, 648.0, '20']]

获取完数据，开始写入统计表。因为统计表是`xlsx`后缀的，这个时候可以叫`openpyxl`出来活动一下筋骨了。顺便导入“设置单元格格式”的相关的模块。因为数据列表`data`中全是子列表，所以直接使用`append`方法整行写入最为方便。数据写完后，需要设置行高，字号，加粗，对齐，单元格边框等。全部设置完后保存数据。

In [16]:
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment #设置单元格格式
thin = Side(border_style="thin", color="000000")#定义边框粗细及颜色

file = "统计表模板.xlsx"
wb = load_workbook(file)
ws = wb.active

#写入数据
for i in data:
    ws.append(i)
    
#设置字号，对齐，缩小字体填充，加边框
#Font(bold=True)可加粗字体
for row_number in range(3, ws.max_row+1):    
    ws.row_dimensions[row_number].height = 25 #设置行高
    for col_number in range(1,5):
        c = ws.cell(row=row_number,column=col_number)
        c.font = Font(size=11,bold=True)
        c.border = Border(top=thin, left=thin, right=thin, bottom=thin)
        c.alignment = Alignment(horizontal="center", vertical="center")
    
wb.save("统计表.xlsx")
    

看一下结果，美美哒。
![](images\theresult.png)